# Cryptocurrency Fundamental Analysis Tool

In [70]:
import json
import requests
import pandas as pd
import hvplot.pandas
import panel as pn
import holoviews as hv
import numpy as np

pn.extension(sizing_mode="stretch_width")

In [89]:

# Coin Metrics API endpoint url
url = 'https://community-api.coinmetrics.io/v4/timeseries/asset-metrics?'

# Dates to get historical data for the last 3 years
start_date = pd.Timestamp("2018-01-01").isoformat()
end_date = pd.Timestamp.today().isoformat()

# Number of records to retrieve
page_size = 6000

# Define the list of crypto assets to analyze
crypto_asset = ['btc', 'eth', 'doge', 'usdt']

def get_data(metric):
    
    # Save the JSON response as lists
    symbol = []
    time = []
    value = []

    # Loop through the selected tokens
    for token in crypto_asset:
        
        # Build URL with the corresponding token, metric, and dates
        get_url = f"{url}&assets={token}&metrics={metric}&start_time={start_date}&end_time={end_date}&page_size={page_size}"

        # Get the response from Coin Metrics API is JSON format
        response = requests.get(get_url).json()

        # Iterate through the response to add the relevant items to its corresponding list
        for item in response['data']:

            symbol.append(item['asset'])
            time.append(item['time'])
            value.append(item[metric])
            
    # Build the dataframe with the data
    data_df = pd.DataFrame(
    {'Token': symbol,
     'Date': time,
     'Value': value
    })
    
    # Format the date for easy reading 
    data_df['Date'] = pd.to_datetime(data_df['Date'], format="%Y/%m/%d").dt.date
    
    # Change the value type to 'float' to allow graphing
    data_df['Value'] = data_df['Value'].astype(float)
    
    # Format the DataFrame to allow xgraphing
    updated_data_df = data_df.pivot_table('Value', ['Date'], 'Token')
    
    return updated_data_df


In [90]:
# Create a dataframe for each metric by calling the get_data function and passing the relevant Coin Metric reference
prices = get_data('ReferenceRate')
active_addresses = get_data('AdrActCnt')
free_float_supply = get_data('SplyFF')
transaction_count = get_data('TxCnt')
market_cap_total = get_data('CapMrktCurUSD')
market_cap_real = get_data('CapRealUSD')
mk_value_percent = get_data('CapMVRVCur')
addresses_1 = get_data('AdrBalUSD1Cnt')
addresses_1m = get_data('AdrBalUSD1MCnt')



#Rolling data doesn't work in plot below


In [85]:
addresses_1m['eth'].hvplot()

:Curve   [Date]   (eth)

In [75]:
prices.tail(1)

Token,bnb,btc,doge,eth
Date,,,,
2021-08-16,413.841375,47074.37071,0.339528,3310.727393


In [91]:
# Create price visualization widget using https://panel.holoviz.org/gallery/apis/stocks_hvplot.html

title = '## Price History'

# Define the price plots
def get_plot_price(ticker, window_size):
    df = prices[ticker]
    return df.hvplot.line('Date', grid=True, responsive=True, height=300, title=f"{ticker} prices")

In [92]:
# Create a dynamic plot with the price information

interact = pn.interact(get_plot_price, ticker=crypto_asset, window_size=(1, 21, 5))

price = pn.Row(
    pn.Column(title, interact[0], sizing_mode="fixed", width=300),
    interact[1]
)

price

Row(sizing_mode='stretch_width')
    [0] Column(sizing_mode='fixed', width=300)
        [0] Markdown(str, sizing_mode='stretch_width')
        [1] Column(sizing_mode='stretch_width')
            [0] Select(name='ticker', options=['btc', 'eth', ...], sizing_mode='stretch_width', value='btc')
            [1] IntSlider(end=21, name='window_size', sizing_mode='stretch_width', start=1, step=5, value=11)
    [1] Row(sizing_mode='stretch_width')
        [0] HoloViews(Curve, name='interactive14982', sizing_mode='stretch_width')

In [93]:
title = '## Active Addresses'

# Define the price plots
def get_plot_addresses(ticker, window_size):
    df = active_addresses[ticker]
    return df.hvplot.line('Date', grid=True, responsive=True, height=300, title=f"{ticker} addresses")

In [94]:
# Create a dynamic plot with the addresses information

interact = pn.interact(get_plot_addresses, ticker=crypto_asset, window_size=(1, 21, 5))

addresses = pn.Row(
    pn.Column(title, interact[0], sizing_mode="fixed", width=300),
    interact[1]
)

addresses

Row(sizing_mode='stretch_width')
    [0] Column(sizing_mode='fixed', width=300)
        [0] Markdown(str, sizing_mode='stretch_width')
        [1] Column(sizing_mode='stretch_width')
            [0] Select(name='ticker', options=['btc', 'eth', ...], sizing_mode='stretch_width', value='btc')
            [1] IntSlider(end=21, name='window_size', sizing_mode='stretch_width', start=1, step=5, value=11)
    [1] Row(sizing_mode='stretch_width')
        [0] HoloViews(Curve, name='interactive15340', sizing_mode='stretch_width')

In [ ]:
# Creat a fancy dashboard

ticker = pn.widgets.Select(name='Tokens', options=crypto_asset)
window = pn.widgets.IntSlider(name='Window Size', value=5, start=1, end=21)

row = pn.Row(
    pn.Column(title, ticker, window, sizing_mode="fixed", width=300),
    get_plot(ticker.options[0], window.value)
)

def update(event):
    row[1].object = get_plot(ticker.value, window.value)

    
ticker.param.watch(update, 'value')
window.param.watch(update, 'value')

row

#pn.template.FastListTemplate(site="Dashboard", title="Crypto Fundamental Analysis Tool", sidebar=[ticker, window], main=[row[1]]).show()

template = pn.template.FastListTemplate(site="Dashboard", title="Crypto Fundamental Analysis Tool")
template.sidebar.append(ticker)
template.sidebar.append(window)

template.main.append(row[1])
template.show()

In [ ]:
# To show side-by-side graphs: https://panel.holoviz.org/reference/templates/FastListTemplate.html